In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import snntorch as snn
from snntorch import utils
from snntorch import spikegen
import snntorch.spikeplot as splt
from snntorch import surrogate
from snntorch import functional

import matplotlib.pyplot as plt
import numpy as np
from trainer import *

import torchvision
from torchvision import models

In [2]:


dtype=torch.float
torch.manual_seed(734)
print("Feedforward SNN Trained on cifar")

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Training Parameters
batch_size=256
data_path='./tmp/data/cifar10/'
num_classes = 10  # cifar has 10 output classes

# Define a transform
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

cifar_train = datasets.CIFAR10(data_path, train=True, download=True,transform=transform)
cifar_test = datasets.CIFAR10(data_path, train=False, download=True,transform=transform)

train_loader = DataLoader(cifar_train, batch_size=batch_size, shuffle=True,drop_last=False,pin_memory=True,num_workers=1)
test_loader = DataLoader(cifar_test, batch_size=batch_size, shuffle=True,drop_last=False,pin_memory=True,num_workers=1)

print("Train batches:",len(train_loader))
print("Test batches:",len(test_loader))

Feedforward SNN Trained on cifar
Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Train batches: 196
Test batches: 40


Useful to know what a convolutional layers output dimension is given by $n_{out}=\frac{n_{in}+2p-k}{s}+1$, 
with default padding=0 stride=1 $n_{out}=n_{in}-k+1$.
For max pool with defaults, $\lfloor\frac{n}{2}\rfloor$.

In [3]:
################ VGG9 Model #############################

# layer parameters

lr=5e-4

class Net(nn.Module):

    def __init__(self,c = None):
        super().__init__()

        self.loss = nn.CrossEntropyLoss()
        self.accuracy_metric = lambda logits, targets: (torch.argmax(logits,dim=1) == targets).sum().item() / targets.size(0)

        self.layers = nn.ModuleList([
            nn.Conv2d(3,32,3,padding=1),
            nn.Conv2d(32,32,3,padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(64,64,3,padding=1),
            nn.Conv2d(64,64,3,padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(128,128,3,padding=1),
            nn.Conv2d(128,128,3,padding=1),
            nn.MaxPool2d(2),
            nn.Linear(4096,128),
            nn.Linear(4096,11),      
        ])

        

        self.neurons = nn.ModuleList([
            nn.ReLU()
        ] * len(self.layers))

        self.to(device)
        
    def forward(self,x):
        for i in range(len(self.layers)):
            if(i==9): #need to flatten from pooling to Linear
                x= torch.flatten(x,start_dim=1)

            x = self.neurons[i]( self.layers[i](x) )
        return x
    
###################################################################################
    






In [4]:
model_path = "./models/SNN_VGG_CFIAR10.pt"
net = Net()
optimiser = torch.optim.Adam(net.parameters(),lr=lr,weight_decay=0)
gen_reset() #reset the PRNG generators for the random samplers so we consistently get the same sequence of samples for each experiment run
net = trainer(net,train_loader=train_loader,valid_loader=test_loader,model_path=model_path,optimiser=optimiser,epochs=20,iterations=None,valid_after=250,valid_iterations=1,deepr=False,device=device)
gen_reset()
a = test_stats(net,test_loader=test_loader,iterations=None,device=device)

Training progress::   0%|          | 0/9800 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x2048 and 4096x4096)

: 